## Download & install packages and models

In [1]:
import sys
sys.path.append('..')

In [2]:
from typing import List
import sentence_embedding_evaluation_german as seeg
import torch
import numpy as np

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%%capture
!pip install fasttext

In [4]:
# !wget -nc "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz"
# !gzip -d cc.de.300.bin.gz

In [5]:
%%capture
import fasttext
import fasttext.util
fasttext.util.download_model('de', if_exists='ignore')

## Specify & run sentence embedding evaluation

In [6]:
%%time
# (1) Instantiate your Embedding model
ft = fasttext.load_model('cc.de.300.bin')

# (2) Specify the preprocessing
def preprocesser(batch: List[str], params: dict=None) -> List[List[float]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[float]]
        A list of embedding vectors
    """
    features = np.vstack([
        ft.get_sentence_vector(str(sent).replace("\n", " ")) for sent in batch])
    return features

# (3) Training settings
params = {
    'datafolder': '../datasets',
    'bias': True,
    'balanced': True,
    'batch_size': 128, 
    'num_epochs': 500,
    # 'early_stopping': True,
    # 'split_ratio': 0.1,  # if early_stopping=True
    # 'patience': 10,  # if early_stopping=True
}

# (4) Specify downstream tasks
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'ARCHI', 'LSDC']

# (5) Run experiments
results = seeg.evaluate(downstream_tasks, preprocesser, **params)

CPU times: user 18min 57s, sys: 1min 44s, total: 20min 41s
Wall time: 21min 36s


In [7]:
# save results
import json
with open("seeg-results-fasttext.json", 'w') as fp:
    json.dump(results, fp, indent=2)

## Display the results

In [8]:
print("Task | Epochs | N train | N test")
for res in results:
    print(f"{res['task']:>7s}: {res['epochs']:5d} {res['train']['num']:6d} {res['test']['num']:6d}")

Task | Epochs | N train | N test
 FCLAIM:   500   3244    944
   VMWE:   500   6652   1447
 OL19-C:   500   1921    930
 ABSD-2:   500  19432   2555
  MIO-P:   500   4668   4668
  ARCHI:   500  18809   4743
   LSDC:   500  74140   8602


In [9]:
metric = 'acc'  # 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.671  0.661
   VMWE:  0.857  0.833
 OL19-C:  0.770  0.757
 ABSD-2:  0.648  0.643
  MIO-P:  0.793  0.789
  ARCHI:  0.604  0.477
   LSDC:  0.712  0.568


In [10]:
metric = 'acc-balanced'  # 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.678  0.660
   VMWE:  0.768  0.735
 OL19-C:  0.736  0.675
 ABSD-2:  0.650  0.580
  MIO-P:  0.829  0.824
  ARCHI:  0.603  0.477
   LSDC:  0.408  0.404


In [11]:
metric = 'f1'  # 'f1', 'f1-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.671  0.661
   VMWE:  0.857  0.833
 OL19-C:  0.770  0.757
 ABSD-2:  0.648  0.643
  MIO-P:  0.793  0.789
  ARCHI:  0.604  0.477
   LSDC:  0.712  0.568


In [12]:
metric = 'f1-balanced'  # 'f1', 'f1-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.658  0.643
   VMWE:  0.758  0.725
 OL19-C:  0.650  0.623
 ABSD-2:  0.559  0.515
  MIO-P:  0.729  0.725
  ARCHI:  0.603  0.472
   LSDC:  0.357  0.334


In [13]:
# class label distributions (inference)
for res in results:
    print(res['task'], res['test']['num'])
    print(res['test']['distr-test'])
    print(res['test']['distr-pred'])

FCLAIM 944
{'0': '630', '1': '314'}
{'1': '418', '0': '526'}
VMWE 1447
{'1': '256', '0': '1191'}
{'1': '283', '0': '1164'}
OL19-C 930
{'0': '796', '1': '134'}
{'0': '688', '1': '242'}
ABSD-2 2555
{'1': '1670', '0': '780', '2': '105'}
{'0': '989', '1': '1266', '2': '300'}
MIO-P 4668
{'1': '812', '0': '3856'}
{'1': '1598', '0': '3070'}
ARCHI 4743
{'2': '1177', '3': '1176', '1': '1199', '0': '1191'}
{'2': '1395', '1': '1414', '3': '810', '0': '1124'}
LSDC 8602
{'13': '280', '6': '346', '12': '940', '3': '925', '2': '944', '1': '934', '11': '931', '0': '453', '10': '511', '5': '924', '4': '65', '8': '923', '9': '83', '7': '343'}
{'8': '1159', '11': '990', '12': '1235', '5': '1231', '3': '1779', '0': '891', '1': '1316', '7': '1'}
